Bankruptcy Prediction Analysis

In [ ]:
import pandas as pd
import os

os.getcwd()

#data_url = 'https://raw.githubusercontent.com/sowide/bankruptcy_dataset/main/american_bankruptcy_dataset.csv'
#df = pd.read_csv(data_url)

from google.colab import drive
drive.mount('/content/drive')

#Test

# Now you can access files from your Google Drive using paths like "/content/drive/My Drive/path_to_your_file"


#df = pd.read_csv('Data/american_bankruptcy.csv')

'/content'

In [ ]:
df.head(5)


In [ ]:
df = df.rename(columns={'X1':'ca','X2':'cogs','X3':'d_and_a','X4':'ebitda','X5':'inventory','X6':'ni','X7':'ar','X8':'mv','X9':'ns','X10':'ta','X11':'tltd','X12':'ebit','X13':'gp','X14':'cl','X15':'re','X16':'tr','X17':'tl','X18':'toe'})


In [ ]:
df['wc'] = df['ca'] - df['cl']
df['wc_to_ta'] = df['wc'] / df['ta']
df['re_to_ta'] = df['re'] / df['ta']
df['ebit_to_ta'] = df['ebit'] / df['ta']
df['mv_to_tl'] = df['mv'] / df['tl']
df['tr_to_ta'] = df['tr'] / df['ta']

df['z_score'] = 1.2*df['wc_to_ta'] + 1.4*df['re_to_ta'] + 3.3*df['ebit_to_ta'] + 0.6*df['mv_to_tl'] + 1.0*df['tr_to_ta']



In [ ]:
df['latest_period'] = df['fyear'] == df.groupby('company_name')['fyear'].transform('max')
df['latest_period'] = df['latest_period'].astype(int)

In [ ]:
df['default'] = ((df['latest_period'] == 1) & (df['status_label'] == 'failed')).astype(int)


In [ ]:
df['default'].value_counts()

default
0    78073
1      609
Name: count, dtype: int64